In [103]:
import json
import yaml
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from copy import deepcopy
import random
random.seed(0)

In [60]:
train_data_original = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final.json', 'r'))

In [61]:
train_data_original[0]

{'id': 'Step 7: Can you provide me with a random Chuck Norris joke? I also want to know a fascinating mathematical property about the number 1729. Please find something interesting about it.',
 'conversations': [{'from': 'system',
   'value': 'You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can\'t go back to one of the former state, if you want to restart the task, say "I give up 

In [62]:
def get_tool_set(item):
    tool_system_msg = "You have access of the following tools:\n"
    api_system_msg = "\nSpecifically, you have access to the following APIs: "

    system_step = item['conversations'][0]['value']

    tool_descriptions = system_step.split(tool_system_msg)[1].split(api_system_msg)[0]
    tools = system_step.split(api_system_msg)[1]

    try:
        tools = yaml.load(tools, yaml.Loader)
        return tools
    except Exception as e:
        return []

In [63]:
train_data_parseble_tools = []
for item in tqdm(train_data_original):
    tools = get_tool_set(item)
    if tools:
        item_parseble_tools = deepcopy(item)
        item_parseble_tools['tools'] = tools
        train_data_parseble_tools.append(item_parseble_tools)

  0%|          | 0/12482 [00:00<?, ?it/s]

In [64]:
len(train_data_parseble_tools)

11038

In [65]:
get_tool_set(train_data_original[0])

[{'name': 'jokes_search_for_chuck_norris',
  'description': 'This is the subfunction for tool "chuck_norris", you can use this tool.The description of this function is: "Free text search"',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'The search term.'}},
   'required': ['query'],
   'optional': []}},
 {'name': 'jokes_random_for_chuck_norris',
  'description': 'This is the subfunction for tool "chuck_norris", you can use this tool.The description of this function is: "Retrieve a random chuck joke in JSON format."',
  'parameters': {'type': 'object',
   'properties': {},
   'required': [],
   'optional': []}},
 {'name': 'jokes_categories_for_chuck_norris',
  'description': 'This is the subfunction for tool "chuck_norris", you can use this tool.The description of this function is: "Retrieve a list of available joke categories."',
  'parameters': {'type': 'object',
   'properties': {},
   'required': [],
   'optional': []}},
 {'name

In [66]:
all_apis = set()
all_tools = set()
for item in tqdm(train_data_parseble_tools):
    curr_tools = item['tools']
    for tool in curr_tools:
        tool_name = tool['name']
        if 'for' in tool_name:
            all_tools.add(tool_name.split('_for_')[1])
        else:
            all_tools.add(tool_name)
        all_apis.add(tool['name'])

  0%|          | 0/11038 [00:00<?, ?it/s]

In [104]:
def get_tool_names(item):
    tools = item['tools']
    tool_names = set()
    for tool in tools:
        tool_name = tool['name']
        if 'for' in tool_name:
            tool_names.add(tool_name.split('_for_')[1])
        else:
            tool_names.add(tool_name)
    return tool_names

In [112]:
train_tools, val_tools = train_test_split(list(all_tools), test_size=0.4)
train_tools = set(train_tools)
val_tools = set(val_tools)
if 'Finish' in val_tools:
    val_tools.remove('Finish')
    train_tools.add('Finish')
print(len(train_tools), len(val_tools))

intersection_count = 0
for item in tqdm(train_data_parseble_tools):
    tools = get_tool_names(item)
    tools.remove('Finish')
    tools_from_train = tools.intersection(train_tools)
    tools_from_val = tools.intersection(val_tools)
    if tools_from_train and tools_from_val:
        rand = random.uniform(0,1)
        if rand >= 0.4:
            for tool in tools_from_val:
                val_tools.remove(tool)
                train_tools.add(tool)
        else:
            for tool in tools_from_train:
                train_tools.remove(tool)
                val_tools.add(tool)

print(len(train_tools), len(val_tools))

1398 933


  0%|          | 0/11038 [00:00<?, ?it/s]

1629 702


In [115]:
intersection_count = 0
train_count = 0
val_count = 0

train_sample = []
val_sample = []
for item in tqdm(train_data_parseble_tools):
    tools = get_tool_names(item)
    tools.remove('Finish')
    tools_from_train = tools.intersection(train_tools)
    tools_from_val = tools.intersection(val_tools)
    if tools_from_train:
        train_count += 1
    if tools_from_val:
        val_count += 1
    if tools_from_train and tools_from_val:
        intersection_count += 1

    if tools_from_train and not tools_from_val:
        train_sample.append(item)
    else:
        val_sample.append(item)
        
print(train_count, val_count, intersection_count)
print(len(train_sample))
print(len(val_sample))

  0%|          | 0/11038 [00:00<?, ?it/s]

8860 3276 1098
7762
3276


In [128]:
print(len(train_data_original))
print(len(train_data_parseble_tools))
print(len(train_sample))
print(len(val_sample))
print(len(train_sample) + len(val_sample))

12482
11038
7762
3276
11038


In [118]:
def split_data_last_tool_call(data):
    data_splitted = []
    for item in data:
        conversations = item['conversations']
        for i in range(len(conversations)):
            if conversations[i]['from'] == 'assistant':
                splitted_item = deepcopy(item)
                splitted_item['conversations'] = splitted_item['conversations'][:i+1]
                data_splitted.append(splitted_item)
    return data_splitted

In [126]:
train_original_splitted = split_data_last_tool_call(train_data_original)
len(train_original_splitted)

40605

In [119]:
train_sample_splitted = split_data_last_tool_call(train_sample)
val_sample_splitted = split_data_last_tool_call(val_sample)

In [122]:
len(train_sample_splitted)

25514

In [123]:
len(val_sample_splitted)

10322

In [120]:
json.dump(train_sample_splitted, open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final_train_train.json', 'w'))

In [121]:
json.dump(val_sample_splitted, open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final_train_val.json', 'w'))

In [131]:
len(train_sample)

7762

In [132]:
len(val_sample)

3276

In [133]:
json.dump(train_sample, open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final_train_train_full_chains.json', 'w'))

In [134]:
json.dump(val_sample, open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_undetectable_errors_final_train_val_full_chains.json', 'w'))